# Setup

In [1]:
from pathlib import Path
import platform
import os
import shutil
import time
import matplotlib.pylab as plt

import spikeinterface.sorters as ss
import spikeinterface.full as si

# may need to run pip install

# Add path to kilosort2 and ironclust repos
si.Kilosort2Sorter.set_kilosort2_path(Path.resolve(Path('../../Kilosort-2.0')))
si.IronClustSorter.set_ironclust_path(Path.resolve(Path('../../ironclust')))

print(ss.installed_sorters())
print(f"SpikeInterface version: {si.__version__}")

Setting KILOSORT2_PATH environment variable for subprocess calls to: D:\hannah\Dropbox\code\spikesort\kilosort-2.0
Setting IRONCLUST_PATH environment variable for subprocess calls to: D:\hannah\Dropbox\code\spikesort\ironclust
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptsbkqfb3r\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptjuuaf_p8\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptrmh70el4\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptxxel0j3s\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptcuumf351\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptrwzfmqj_\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptm0ldp10z\script.bat
RUNNING SHELL SCRIPT: C:\Users\Hannah\AppData\Local\Temp\tmp_shellscriptn6npk9i8\script.bat
['ironclust', 'kilosort2', 'spykingci

Set paths that are used throughout: results and data

In [2]:
results_path = Path.resolve(Path('..','results')) 
print(results_path)
data_path = results_path/'toy_example_recording'
print(data_path)


D:\hannah\Dropbox\code\spikesort\spikesort-hp\results
D:\hannah\Dropbox\code\spikesort\spikesort-hp\results\toy_example_recording


Make toy example recording data in the results folder:

In [3]:
if not data_path.exists():
    rec, sorting = si.toy_example(num_segments=1, duration=100, seed=1, num_channels=16, num_columns=2)
    rec.save(folder=data_path)
    print('Created toy data')
else:
    print('Toy data already exists')
print(data_path)

Toy data already exists
D:\hannah\Dropbox\code\spikesort\spikesort-hp\results\toy_example_recording


Load the recording extractor

In [4]:
rec = si.load_extractor(data_path)
rec

BinaryFolderRecording: 16 channels - 1 segments - 30.0kHz - 100.000s

# Run spikesorting
Run Kilosort on the test data: 

Note: make sure your startup.m file in Matlab doesn't change the current directory!

In [ ]:
# Check out kilosort params
si.get_default_sorter_params('kilosort2')

Change Kilosort2 params

In [ ]:
# Change params
default_kilosort2_params = si.Kilosort2Sorter.default_params()
print(default_kilosort2_params)
#default_kilosort2_params['freq_min'] = 300
# default_kilosort2_params['minFR'] = 0
# default_kilosort2_params['minfr_goodchannels'] = 0
# default_kilosort2_params['keep_good_only'] = True

# Can't use these?
#default_kilosort2_params['reorder'] = 0 
#default_kilosort2_params['CAR'] = 1

print(default_kilosort2_params)


In [ ]:
# Run sorter
sorter_name = 'kilosort2'
output_folder = results_path/(sorter_name+'_output')
print(output_folder)

sorting = si.run_sorter(sorter_name, rec, output_folder, verbose=True, remove_existing_folder=True, **default_kilosort2_params)

# Get waveforms
waveform_folder = results_path/(sorter_name+'_waveforms')
print(waveform_folder)
si.extract_waveforms(rec, sorting, waveform_folder,
    n_jobs=1, total_memory="10M", max_spikes_per_unit=500, return_scaled=False, overwrite=True)

# Export to Phy
we = si.WaveformExtractor.load_from_folder(waveform_folder)
phy_folder =  results_path/(sorter_name+'_phy')
print(phy_folder)
si.export_to_phy(we, output_folder=phy_folder, verbose=False, remove_if_exists=True)

print(sorting)

In [ ]:
# Run sorter
sorter_name = 'ironclust'
output_folder = results_path/(sorter_name+'_output')
print(output_folder)
sorting = si.run_sorter(sorter_name, rec, output_folder, verbose=True, remove_existing_folder=True)

# Get waveforms
waveform_folder = results_path/(sorter_name+'_waveforms')
print(waveform_folder)
si.extract_waveforms(rec, sorting, waveform_folder,
    n_jobs=1, total_memory="10M", max_spikes_per_unit=500, return_scaled=False, overwrite=True)

# Export to Phy
we = si.WaveformExtractor.load_from_folder(waveform_folder)
phy_folder = results_path/(sorter_name+'_phy')
print(phy_folder)
si.export_to_phy(we, output_folder=phy_folder, verbose=False, remove_if_exists=True)

# Post-processing
Calculate quality metrics for each sorter

In [ ]:
# Load waveforms
we_ks = si.WaveformExtractor.load_from_folder(results_path/'kilosort2_waveforms')
we_ic = si.WaveformExtractor.load_from_folder(results_path/'ironclust_waveforms')

# Compute quality metrics
metric_names=['snr', 'isi_violation', 'amplitude_cutoff','isolation_distance']
metrics_ks = si.compute_quality_metrics(we_ks, metric_names)
metrics_ic = si.compute_quality_metrics(we_ic, metric_names)
print(metrics_ks)
print(metrics_ic)

#assert 'isolation_distance' in metrics_ks.columns
# not sure why isolation distance isn't showing up for ic


# Compare sorters

In [5]:
# Load sorting
sorting_ks = si.read_phy(results_path/'kilosort2_phy')
sorting_ic = si.read_phy(results_path/'ironclust_phy')

comp_ks_ic = si.compare_two_sorters(sorting1=sorting_ks, sorting2=sorting_ic)
comp_multi = si.compare_multiple_sorters(sorting_list=[sorting_ks, sorting_ic],
                                         name_list=['ks', 'ic'])

print('Relative to Kilosort2:')
print(comp_ks_ic.hungarian_match_12)

print('Relative to Ironclust:')
print(comp_ks_ic.hungarian_match_21)


Relative to Kilosort2:
0     1.0
1     2.0
2    -1.0
3     3.0
4     4.0
5    -1.0
6     5.0
7     6.0
8     7.0
9     8.0
10   -1.0
dtype: float64
Relative to Ironclust:
0   -1.0
1    0.0
2    1.0
3    3.0
4    4.0
5    6.0
6    7.0
7    8.0
8    9.0
dtype: float64


In [6]:
sorting_ks = si.read_phy(results_path/'kilosort2_phy')
sorting_ic = si.read_phy(results_path/'ironclust_phy')

sorting_agreement = comp_multi.get_agreement_sorting(minimum_agreement_count=2)
print('Units in agreement:', sorting_agreement.get_unit_ids())
# w_multi = si.plot_multicomp_graph(comp_multi)
# plt.show() # not working correctly

Units in agreement: [0 1 3 4 6 7 8 9]


In [ ]:
for ii in range(0,40):
    try:
        os.close(ii)
        print('Closed a file!')
    except:
        print('did nothing')


Export consensus agreement to Phy

In [9]:
# Get waveforms
waveform_folder = results_path/'consensus_waveforms2'
print(waveform_folder)

# multisortingcomparison=True
si.extract_waveforms(rec, sorting_agreement, waveform_folder,
    n_jobs=1, total_memory="10M", max_spikes_per_unit=500, return_scaled=False, overwrite=True)


D:\hannah\Dropbox\code\spikesort\spikesort-hp\results\consensus_waveforms2


WaveformExtractor: 16 channels - 8 units - 1 segments
  before:90 after:120 n_per_units:500

In [14]:

# Export to Phy
we = si.WaveformExtractor.load_from_folder(waveform_folder)

TypeError: __init__() missing 2 required positional arguments: 'sampling_frequency' and 'multisortingcomparison'

In [ ]:

phy_folder = results_path/'consensus_phy'
print(phy_folder)
si.export_to_phy(we, output_folder=phy_folder, verbose=False, remove_if_exists=True)